In [1]:
from langchain_google_genai import GoogleGenerativeAI
from dotenv import load_dotenv

/home/saqib736/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [47]:
load_dotenv()
llm = GoogleGenerativeAI(model="gemini-pro", temperature=1)

In [48]:
result = llm.invoke('hi')
print(result)

Hello there! How can I help you today?


In [114]:
import re

def CleanOutput(input_str: str) -> str:
    matches = re.findall(r"-?\d+\.\d+|-?\d+", input_str)
    if matches:
        return matches[0]
    else:
        return ''

print(CleanOutput("Decimal('22899.800000')"))             

22899.800000


In [154]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.prompts import PromptTemplate

In [ ]:
db_user = "wsluser"
db_password = "saqungra736"
db_host = "172.30.160.1"
db_name = "mystore_tshirts"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)

print(db.table_info)

In [155]:
custom_prompt = PromptTemplate(
    input_variables=["input", "table_info", "dialect"],
    template="""
            You are an assistant that helps users answer questions by querying a SQL database.

            Given an input question, first create a syntactically correct {dialect} SQL query to run, then execute the query and look at the results, and then return the answer to the user.

            Use the following format:

            Question: "Question here"

            SQLQuery: SQL Query to run

            SQLResult: Result of the SQLQuery

            Answer: it should be same as SQLResult

            Do not include any Markdown formatting like ```sql or ``` in your query or results.

            Only use the tables listed below.

            {table_info}

            Question: {input}
            """,
)

db_chain = SQLDatabaseChain.from_llm(llm, db, prompt=custom_prompt, verbose=True)


In [156]:
qns1 = db_chain("How many t-shirts do we have left for nike in extra small size and white color?")



> Entering new SQLDatabaseChain chain...
How many t-shirts do we have left for nike in extra small size and white color?
SQLQuery:SELECT stock_quantity, brand, color, size
FROM t_shirts
WHERE brand = 'Nike' AND color = 'White' AND size = 'XS';
SQLResult: [(43, 'Nike', 'White', 'XS')]
Answer:43
> Finished chain.


In [157]:
qns1

{'query': 'How many t-shirts do we have left for nike in extra small size and white color?',
 'result': '43'}

In [158]:
if isinstance(qns1, dict):
    qns1 = CleanOutput(qns1['result'])
else:
    qns1 = CleanOutput(qns1) 
print(qns1)

43


In [131]:
qns2 = db_chain("How much is the price of the inventory for all small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT SUM(price * stock_quantity) AS total_inventory_value
FROM t_shirts
WHERE size = "S"
SQLResult: [(Decimal('17315'),)]
Answer:Decimal('17315')
> Finished chain.


In [101]:
qns2

{'query': 'How much is the price of the inventory for all small size t-shirts?',
 'result': '17315'}

In [132]:
if isinstance(qns2, dict):
    qns2 = CleanOutput(qns2['result'])
else:
    qns2 = CleanOutput(qns2) 
print(qns2)

17315


In [133]:
qns3 = db_chain.run("If we have to sell all the Levi T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Levi T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?
SQLQuery:SELECT SUM((1 - d.pct_discount / 100) * t.price * t.stock_quantity) AS revenue
FROM t_shirts t
JOIN discounts d ON t.t_shirt_id = d.t_shirt_id
WHERE t.brand = 'Levi';
SQLResult: [(Decimal('5994.800000'),)]
Answer:Decimal('5994.800000')
> Finished chain.


Got Wrong Answer above

In [134]:
sql_code = """
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """

qns3 = db_chain.run(sql_code)



> Entering new SQLDatabaseChain chain...

select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 
SQLQuery:select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('22899.800000'),)]
Answer:Decimal('22899.800000')
> Finished chain.


In [135]:
qns3

"Decimal('22899.800000')"

In [136]:
if isinstance(qns3, dict):
    qns3 = CleanOutput(qns3['result'])
else:
    qns3 = CleanOutput(qns3) 
print(qns3)

22899.800000


In [139]:
qns4 = db_chain.run("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('24788'),)]
Answer:(Decimal('24788'),)
> Finished chain.


In [140]:
qns4

"(Decimal('24788'),)"

In [141]:
if isinstance(qns4, dict):
    qns4 = CleanOutput(qns4['result'])
else:
    qns4 = CleanOutput(qns4) 
print(qns4)

24788


In [147]:
qns5 = db_chain.run("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")



> Entering new SQLDatabaseChain chain...
SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('187'),)]
Answer:(Decimal('187'),)
> Finished chain.


In [148]:
qns5

"(Decimal('187'),)"

In [149]:
if isinstance(qns5, dict):
    qns5 = CleanOutput(qns5['result'])
else:
    qns5 = CleanOutput(qns5) 
print(qns5)

187


In [181]:
few_shots = [
    {'Question' : "How many t-shirts do we have left for Nike in XS size and white color?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns1},
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': qns2},
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': qns3} ,
     {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : qns4},
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns5
     }
]

In [182]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

In [183]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
to_vectorize = [" ".join(example.values()) for example in few_shots]

In [163]:
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=few_shots)

In [165]:
example_selector =  SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

In [170]:
mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLResult: Result of the SQLQuery
Answer: Final answer here

No pre-amble.
"""

In [174]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX

In [175]:
custom_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult", "Answer"],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [176]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=custom_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"]
)

In [179]:
new_chain = SQLDatabaseChain.from_llm(llm, db, prompt=few_shot_prompt, verbose=True)

In [184]:
ans1 = new_chain("How many total white color Levi's shirt I have?")



> Entering new SQLDatabaseChain chain...
How many total white color Levi's shirt I have?
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('187'),)]
Answer:187
> Finished chain.


In [187]:
ans2 = new_chain("How much is the price of the inventory for all small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('17315'),)]
Answer:17315
> Finished chain.


In [188]:
ans3 = new_chain("How much is the price of all white color levi t shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of all white color levi t shirts?
SQLQuery:SELECT sum(price * stock_quantity) AS value_white
FROM   t_shirts
WHERE  brand = 'Levi' AND color = 'White' limit 5
SQLResult: [(Decimal('3928'),)]
Answer:3928
> Finished chain.


In [189]:
ans4 = new_chain("If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?
SQLQuery:SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Nike'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('23520.800000'),)]
Answer:23520.800000
> Finished chain.


In [191]:
ans5 = new_chain("If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?
SQLQuery:SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Van Huesen'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('18512.900000'),)]
Answer:18512.900000
> Finished chain.


In [192]:
sql_credential = {  
  'db_user' : "wsluser",
  'db_password' : "saqungra736",
  'db_host' : "172.30.160.1",
  'db_name' : "mystore_tshirts"
}

In [194]:
sql_credential['db_user']

'wsluser'